In [2]:
import fitz
import pandas as pd
import re

FONT_ENGLISH = "AuctionGothicLight"
FONT_KOREAN = "KoPubDotumBold"
FONT_DESC   = "KoPubDotumLight"

def clean_text(text):
    text = re.sub(r'[▶∙❶❷❸❹❺❻❼￭*]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def merge_multiline_english(spans, start_idx):
    """여러 줄로 나뉜 영단어 span들을 병합"""
    first = spans[start_idx]
    eng_parts = [clean_text(first["text"])]
    last_span = first
    j = start_idx + 1

    while j < len(spans):
        s = spans[j]
        if FONT_ENGLISH not in s["font"]:
            break
        # 같은 열(x 좌표 비슷) + y 간격이 작으면 같은 단어
        if abs(s["bbox"][0] - first["bbox"][0]) < 20 and 0 < s["bbox"][1] - last_span["bbox"][3] < 20:
            eng_parts.append(clean_text(s["text"]))
            last_span = s
            j += 1
        else:
            break

    return " ".join(eng_parts).strip(), j

def extract_glossary(pdf_path):
    doc = fitz.open(pdf_path)
    all_entries = []

    for page in doc:
        spans = [s for b in page.get_text("dict")["blocks"]
                   for l in b.get("lines", [])
                   for s in l.get("spans", [])]

        i = 0
        while i < len(spans):
            span = spans[i]
            text = clean_text(span["text"])
            if not text:
                i += 1
                continue

            if FONT_ENGLISH in span["font"]:  # 영어 단어 시작
                eng_word, j = merge_multiline_english(spans, i)

                # 한국어 단어 (바로 오른쪽 또는 근처)
                kor_word = ""
                for s in spans[j:j+5]:
                    if FONT_KOREAN in s["font"]:
                        kor_word = clean_text(s["text"])
                        break

                # 설명 (다음 영어 단어 전까지)
                desc_parts = []
                k = j
                while k < len(spans):
                    s = spans[k]
                    if FONT_ENGLISH in s["font"]:
                        break
                    if FONT_DESC in s["font"]:
                        desc_parts.append(clean_text(s["text"]))
                    k += 1
                description = " ".join(desc_parts)

                if eng_word and kor_word and description:
                    all_entries.append({
                        "영단어": eng_word,
                        "한국단어": kor_word,
                        "설명": description
                    })

                i = k  # jump
            else:
                i += 1

    doc.close()
    return pd.DataFrame(all_entries)

# 실행
pdf_file = "2020년 국제해사기구 전문용어집_페이지 편집본.pdf"
df = extract_glossary(pdf_file)

csv_file = "try4.csv"
df.to_csv(csv_file, sep='|', index=False, encoding='utf-8-sig')

print(f"✅ 추출 완료! {len(df)}개의 항목을 '{csv_file}'에 저장했습니다.")
print(df.head(10))


✅ 추출 완료! 625개의 항목을 'try4.csv'에 저장했습니다.
                            영단어      한국단어  \
0                    Credential       신임장   
1                   Secretariat       사무국   
2                      Assembly        총회   
3                       Council       이사회   
4             Secretary-General      사무총장   
5                Council Reform    이사회 개편   
6            Chair / Vice-Chair  의장 / 부의장   
7                           SMC   고위관리위원회   
8            World Maritime Day  세계 해사의 날   
9  International Maritime Prize   국제 해사 상   

                                                  설명  
0  신임장이란 특정인을 외교 사절로 파견하는 취지와 그 사람의 신분을 상대 국가에 통고...  
1  사무국은 조직·단체에서 인사, 총무 등의 운영사무를 담당하는 부서를 지칭함. 국제해...  
2  국제기구에서 총회는 모든 회원국이 모여서 의제를 토의하고 어떠한 사안에 관한 최종 ...  
3  국제해사기구(IMO)의 실질적인 집행기관으로서 역할을 하고 있음. 총회가 최종 의사...  
4  사무국의 업무를 총괄하는 직위로서 국제해사기구(IMO) 수장을 의미함. 역대 IMO...  
5  기존 40개국으로 구성된 IMO 이사국 제도의 개편을 의미하며, 구체적으로 이사국 ...  
6  회의 진행을 총괄하는 직위를 지칭하며 초기에는 의장을 Chairman 명칭을 사용하...  
7  SMC (Senior Management

In [ ]:
df